# Hands-on Lab 5: Sensing Part 2
### EECS 16B: Designing Information Devices and Systems II, Fall 2022

Updated by Mia Mirkovic (2020)

Updated by Rafael Calleja (2021)

Updated by Steven Lu, Kaitlyn Chan (2021)

Updated by Steven Lu (2022)

Updated by Megan Zeng, Mingyang Wang, and Shrey Aeron (2022)


## Table of Contents

* [Part 0: Before you Begin](#part0)
* [Part 1: Bass-ic Filter](#part1)
* [Part 2: Caught in the Midrange](#part2)


# <span style="color:navy">Introduction</span>
-----
Now that we've built the micboard circuitry, we can convert your voice into an electrical signal for the Arduino to read and work with. However, there is also a lot of background noise picked up by the microphone that we don't want to pass to the Arduino. To remove this, we will be building filters to "filter" out frequencies outside of their designated ranges, attenuating them and preventing them from being passed to the Arduino.

## Lab Note
* [Lab Note](https://eecs16b.org/lab/notes/lab_note5.pdf)

Review the lab note. The lab note provides a technical review of the concepts covered in lab as well as theoretical connections. **You should read the lab notes before your lab section.** They serve as a review and will be helpful as a reference during section.


## Pre-Lab

Complete the pre-lab assignment before doing the lab. Submit your answers to the Gradescope assignment "[\[Lab\] Pre-Lab 5: Sensing Part 2](https://www.gradescope.com/courses/414337/assignments/2290857)." Please make sure that you submit to the correct assignment. Pre-Lab 5 is due on **Monday, October 3 at 11:59pm**. No late submissions will be accepted.

<a id='part0'></a>
# <span style="color:navy">Part 0: Before you Begin</span>
-----
### 0.1. Frequency Response of Speaker-Mic Board System
Before we begin the lab, please copy over `freq_Vpp` into the code block below. This was the dictionary of values for the speaker-microphone frequency response that you recorded in Lab 4. We will use this to analyze the frequency response of our joint speaker-microphone and filter system.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sp
from pylab import * 
%matplotlib inline

In [ ]:
# TODO: Copy your values into freq_Vpp from lab 4
freq_Vpp = {
    20: None,
    50: None,
    200: None, 
    300: None, 
    500: None, 
    800: None, 
    1000: None, 
    1250: None, 
    1750: None, 
    2000: None, 
    2500: None, 
    3000: None, 
    4000: None, 
    5000: None, 
    6000: None, 
    7000: None, 
    8000: None
}
freq = sorted(list(freq_Vpp.keys()))
assert len(freq) == 17, "Ensure that you didn't remove any of the frequences :pandacop:"

Vpp = [freq_Vpp[f] for f in freq] 
print("Vpp = ", Vpp)

# We do not have a measurable input signal Vin, so we normalize the maximum of the frequency response to 1.
gain = np.array(Vpp)/max(Vpp)

plt.loglog(freq, gain)
plt.title('log-Gain vs. log-Frequency')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Gain (Volts/Volts)')
plt.show()

### 0.2. Tune Your Mic Board
#### Materials:
- Speaker (phone, laptop, etc.)

Before every lab that uses the mic board, you will need to retune the mic board in case the potentiometer was bumped or otherwise rotated from its "tuned" position. An abridged version of the micboard tuning steps from Lab 4 is shown below.


#### Tasks:
1. Power both of your regulators with 9V using the +25V channel on your DC power supply. Make sure to set the current limit to 0.1A! **DO NOT plug in your batteries**

2. Connect your oscilloscope probe to $V_{out}$ on your mic board. Add a **peak-to-peak** and **frequency** measurement to the oscilloscope.

3.  **Play a pure tone using a [tone generator](https://www.szynalski.com/tone-generator/) in the range (1500 - 2000 Hz)** on your phone at a reasonable volume (leaning towards the soft side). Do NOT use a Youtube video!

4. Turn the potentiometer on the mic board **(CW: decrease amplitude | CCW: increase amplitude)** using a screwdriver (or your student ID card) until you see a sine wave with a **peak-to-peak of around 1.5V to 2V** on the oscilloscope.
    - If you tuned your micboard previously, you shouldn't need to turn the potentiometer a significant amount.

5. **Verify that the frequency of the measured sine wave matches the tone from your phone, and that the sine wave is centered at around 1.65V.**

## Project Breadboard Layout
In the first part of this lab, you will be building a low-pass filter and a non-inverting amplifier on your breadboard. The low-pass filter corresponds to the dark green region on the recommended project breadboard layout below, with the non-inverting amplifier using the same op-amp chip as the mic board biasing circuit (since there are 2 op-amps per op-amp chip). In the second part of this lab, you will be building a high-pass filter but removing it after checkoff, so **make sure you have enough unallocated space available for this lab**.

<img width="700px" src="images/suggested_layout.jpg">

<a id='part1'></a>
# <span style="color:navy">Part 1: *Bass*ic Filter</span>

-----
In this part, you will build a low-pass filter to isolate the frequencies below the provided cutoff frequency. In addition to serving as part of the band-pass filter in this lab, this filter will eventually be used to better sample voice commands in later phases of the project.

### Materials
- Capacitors
- Resistors
- Function generator
- Oscilloscope

## Tasks

### 1.1. Design and Build the Low-Pass Filter

1. Recall the circuit diagram and cutoff frequency equation $f_c$ for a first-order low pass RC filter. Feel free to refer to lecture notes or draw it out if it helps.

2. Choose values for $R$ and $C$ such that your cutoff frequency $f_c$ is roughly 2500 Hz. Don't worry about getting the exact cutoff frequency: being within a few hundred Hz is fine.
    - We chose 2500 Hz because most of the human vocal range falls below 2500 Hz.
    
3. Record your values for $R$ and $C$ in the code below. Write code to calculate $f_c$ given your values for $R$ and $C$.

4. Build the low-pass filter on your breadboard. Don't connect the micboard output to your low pass filter yet!
    - Look in the lab note for the resistor color code chart and capacitor code chart.
    - Remember, you can combine resistors in series/parallel with each other!

#### Notes/Tips:
- Since we have far more resistor values than we have capacitor values, start by choosing an easy capacitor value, like $1\mu F$, $0.1\mu F$, etc. Pick one that lets you choose resistors of at least 1 $k\Omega$ (otherwise, the current draw would make the readings unstable).

In [ ]:
### YOUR CODE HERE
R = ...
C = ...
f_c = ...

print("Low pass filter")
print("===============")
print("Resistor: {} ohms".format(R))
print("Capacitor: {} farads".format(C))
print("Cutoff frequency: {} Hz".format(f_c))

### 1.2. Test the Low-Pass Filter

With the low pass filter built, we'll check that the cutoff frequency is as expected. To test it in isolation from other components, we'll use the function generator to generate a reliable input. 

1. Set your function generator to output a sinusoid with a 1.65V offset, 1V peak-to-peak voltage, and the frequency as the cutoff frequency of your low pass filter. Remember to set the output load to High-Z.
2. Connect the function generator to the input of your low pass filter. **Make sure your mic board output is not connected to your filter**. 
3. Probe the output of your low pass with your oscilloscope. Ensure the gain/attentuation at the cutoff frequency is what you expect by comparing the peak-to-peak values of your input and output sinusoids. 
    - At the filter's cutoff frequency, the output $V_{pp}$ should be $\frac{1}{\sqrt 2} \cdot V_{pp}$. 
4. Additionally try outputting a very low frequency and very high frequency, and see if the output changes as expected.

### 1.3. Plot the Frequency Response of the Low-Pass-Filtered Mic Board

Now that we've verified the low-pass filter's functionality, it's time to connect it to our mic board. 
1. Disconnect your function generator from your circuit. Connect the **output of the mic board** to the input of your filter.
2. Probe the output of your filter with your oscilloscope.

Once you have done that, fill in the cutoff frequency of your low pass filter in the cell below and run it to plot the ideal frequency response of an RC low pass filter. 

In [ ]:
### TODO: Enter your low cutoff frequency below inside the float() call. 
low_cutoff = float() #TODO

lp_p2p_expected = 1/(np.sqrt(1+((np.array(freq)/(low_cutoff))**2)))


plt.plot(freq, lp_p2p_expected)
plt.xlabel("Frequency (Hz)")
plt.ylabel("Output Voltage/Input Voltage")
plt.title("Frequency vs Gain (Low Pass)")
plt.show()

plt.loglog(freq, lp_p2p_expected)
plt.xlabel("Frequency (Hz)")
plt.ylabel("Output Voltage/Input Voltage")
plt.title("log-Frequency vs log-Gain (Low Pass)")
plt.show()

Next, we will measure the frequency response of the low-pass-filtered mic board. For the given frequencies below in `lp_freq_Vpp`,  measure and record the output peak-to-peak voltage values.

In [ ]:
# Record the amplitude of the output sinusoid at each given frequency. 
# Fill in your values here and replace the "None"s with your measured voltage values
lp_freq_Vpp = {
    100: None,
    200: None,
    400: None, 
    600: None, 
    1000: None, 
    2000: None,
    5000: None
}
lp_freqs = sorted(list(lp_freq_Vpp.keys()))
lp_p2p_observed = [lp_freq_Vpp[f] for f in lp_freqs]

lp_gain_observed = np.array(lp_p2p_observed)/np.max(lp_p2p_observed)

plt.xlabel("Frequency (Hz)")
plt.ylabel("Output Voltage/Input Voltage")
plt.title("Frequency vs Gain (Low Pass)")
plt.plot(freq, lp_p2p_expected)
plt.scatter(lp_freqs, lp_gain_observed)
plt.show()

plt.xlabel("Frequency (Hz)")
plt.ylabel("Output Voltage/Input Voltage")
plt.title("log-Frequency vs log-Gain (Low Pass)")
plt.loglog(freq, lp_p2p_expected)
plt.scatter(lp_freqs, lp_gain_observed)
plt.show()

<!-- <span style="color:#075a04"> **Sanity check: Does the frequency response of your low-pass filtered mic board match the frequency response of an ideal low-pass RC filter? Why or why not?** </span> -->

<span style="color:#075a04">  **Note: The frequency response of your low-pass filtered mic board won't match the frequency response of the ideal low-pass RC filter. Think about why this might be the case.** </span>


Now, we will multiply the ideal low-pass filter frequency response (lp_p2p_expected) with our observed speaker-mic board frequency response (the variable called gain). Run the script below to plot the resulting frequency response.

In [ ]:
response_lp_micboard = np.multiply(lp_p2p_expected, gain)

plt.xlabel("Frequency (Hz)")
plt.ylabel("Output Voltage/Input Voltage")
plt.title("Frequency vs Gain (Low pass filtered Micboard)")
plt.plot(freq, response_lp_micboard)
plt.scatter(lp_freqs,lp_gain_observed)
plt.show()

plt.xlabel("Frequency (Hz)")
plt.ylabel("Output Voltage/Input Voltage")
plt.title("log-Frequency vs log-Gain (Low pass filtered Micboard)")
plt.loglog(freq, response_lp_micboard)
plt.scatter(lp_freqs,lp_gain_observed)
plt.show()


<span style="color:#075a04"> **Sanity check: Does the frequency response of your low-pass filtered mic board match the frequency response of the low-pass filter multiplied with our speaker-mic board frequency response? Why or why not?** </span>

### 1.4. Amplify the Low-Pass Signal
After filtering, the signal may not be strong enough to be measured by the Arduino. Therefore, we may need to amplify the signal.

1. Build a non-inverting amplifier as shown in the diagram below on your breadboard using resistor values of $\geq$1k$\Omega$. Choose a gain such that the largest peak-to-peak voltage in `lp_freq_Vpp` is amplified to around 2-2.5V.
    - **Hint:** The equation for the gain of a non-inverting amplifier is $1 + \frac{R_f}{R_1}$ according to the circuit diagram below.
    - If your $V_{pp}$ is already high enough, you can simply build a buffer.
    - Note that $R_1$ is connected to 1.65V rather than GND to preserve our 1.65V offset.
2. Connect the output of your filter to the input of your amplifier.
3. Probe the output of your amplifier to ensure your signal responds to noise and is not railing.

|Non-inverting Amplifier:|Op-Amp Pinout:|
|:-----:|:---------------:|
|<img style="width:500px" src="images/sp22_low_pass.png">|<img style="width:500px" src="images/op_amp_pinout.png">| 

<a id='part2'></a>
# <span style="color:navy">Part 2: Caught in the Midrange</span>

-----
Often times, we're interested in a more narrow frequency range (i.e. 1kHz to 3kHz) than the range of 0 to our low-pass cutoff frequency. In order to accomplish this, we will build what's known as a band-pass filter, which utilizes both a high-pass and low-pass filter to select a range of frequencies to pass. We'll build the high-pass filter in the unallocated space on the breadboard, **because we'll only be keeping the low-pass filter for future project labs.**



#### Materials
- Capacitors
- Resistors
- Speaker
- Oscilloscope probes

## Tasks

### 2.1. Design and Build the High-Pass Filter Circuit

1. Recall the circuit diagram and cutoff frequency equation $f_c$ for a first-order high pass RC filter.

2. Choose values for $R$ and $C$ such that your cutoff frequency $f_c$ is roughly 800 Hz. (Like the Low Pass filter, being within a few hundred Hz is fine.)
    
3. Record your values for $R$ and $C$ in the code below. Write code to calculate $f_c$ given your values for $R$ and $C$.

4. Build the high-pass filter in an empty spot on your breadboard (you will be removing it after checkoff, so feel free to build it in any open spot).
    - <span style="color:red"> **NOTE: Instead of connecting the end of the resistor to ground, connect it to the `Half Rail`, as shown below.**</span>
    
<img style="width:250px" src="images/high_pass_alone.png">
    
- <span style="color:#075a04"> **Sanity check: Why do we do this?** </span>

#### Notes/Tips:
 
- Since we have far more resistor values than we have capacitor values, start by choosing an easy capacitor value. Pick one that that lets you choose resistors of at least $1k\Omega$.

In [ ]:
### YOUR CODE HERE
R = ...
C = ...
f_c = ...

print("High pass filter")
print("================")
print("Resistor: {} ohms".format(R))
print("Capacitor: {} farads".format(C))
print("Cutoff frequency: {} Hz".format(f_c))

### 2.2. Test the High-Pass Filter

Similar to the low-pass filter, we'll test the high-pass filter with the function generator.

1. Set your function generator to output a sinusoid with a 1.65V offset, 1V peak-to-peak voltage, and the frequency as the cutoff frequency of your high pass filter. 
2. Connect the function generator to the input of your high pass filter.
3. Probe the output of your high pass with your oscilloscope. Ensure the gain/attentuation at the cutoff frequency is what you expect.
4. Additionally try outputting a very low frequency and very high frequency, and see if the output changes as expected.

### 2.3 Build the Band-Pass Filter
1. Connect the **output of the amplifier** you built for the low-pass filter in Part 1.4 to the input of the high pass filter you built in Part 2.1.
    - This amplifier additionally serves the purpose of isolating the low-pass filter from the high-pass filter, removing the need for another buffer between the amplifier's output and the high pass input.
2. At this point, your circuits should be chained as follows:
<img style="width:750px" src="images/band-flow.png">

### 2.4. Plot the Frequency Response of the Micboard Band-Pass System

Fill in your cutoff frequency for your high pass filter in the cell below and run it to plot the frequency response of an ideal band-pass filter. 

In [ ]:
### TODO: Enter your high cutoff frequency below inside the float() call. 
high_cutoff = float() #TODO

hp_p2p_expected =  (np.array(freq)/high_cutoff)/(np.sqrt(1 + (np.array(freq)/high_cutoff)**2))
bp_p2p_expected = np.multiply(hp_p2p_expected, lp_p2p_expected)

plt.xlabel("Frequency (Hz)")
plt.ylabel("Output Voltage/Input Voltage")
plt.title("Frequency vs Gain (Band-Pass)")
plt.plot(freq, bp_p2p_expected)
plt.show()

plt.xlabel("Frequency (Hz)")
plt.ylabel("Output Voltage/Input Voltage")
plt.title("log-Frequency vs log-Gain (Band-Pass)")
plt.loglog(freq, bp_p2p_expected)
plt.show()

Next, probe the output of your band-pass filter. For the given frequencies in `bp_freq_Vpp`, Measure and record the peak-to-peak voltage values. This is the frequency response of your band-pass filtered mic board.

In [ ]:
# Record the amplitude of the output sinusoid at each given frequency. 
# Fill in your values here and replace the "None"s with your measured voltage values
bp_freq_Vpp = {
    200: None,
    500: None, 
    1000: None, 
    2000: None,
    5000: None,
    10000: None
}
bp_freqs = sorted(list(bp_freq_Vpp.keys()))
bp_p2p_observed = [bp_freq_Vpp[f] for f in bp_freqs]

bp_gain_observed = np.array(bp_p2p_observed)/np.max(bp_p2p_observed)

plt.xlabel("Frequency (Hz)")
plt.ylabel("Output Voltage/Input Voltage")
plt.title("Frequency vs Gain (Band Pass)")
plt.plot(freq, bp_p2p_expected)
plt.scatter(bp_freqs, bp_gain_observed)
plt.show()

plt.xlabel("Frequency (Hz)")
plt.ylabel("Output Voltage/Input Voltage")
plt.title("log-Frequency vs log-Gain (Band Pass)")
plt.loglog(freq, bp_p2p_expected)
plt.scatter(bp_freqs, bp_gain_observed)
plt.show()

Now, we will multiply the band-pass filter frequency response (bp_p2p_expected) with our speaker-mic board frequency response (the variable called gain). Run the script below to plot the resulting frequency response.

In [ ]:
response_bp_micboard = np.multiply(bp_p2p_expected, gain)

plt.xlabel("Frequency (Hz)")
plt.ylabel("Output Voltage/Input Voltage")
plt.title("Frequency vs Gain (Band pass filtered Micboard)")
plt.plot(freq, Vpp, label="Micboard")
plt.plot(freq, response_bp_micboard, label="BP Expected")
plt.scatter(bp_freqs, bp_p2p_observed, label="BP observed")
plt.legend()
plt.show()

plt.xlabel("Frequency (Hz)")
plt.ylabel("Output Voltage/Input Voltage")
plt.title("log-Frequency vs log-Gain (Band pass filtered Micboard)")
plt.loglog(freq, Vpp, label="Micboard")
plt.loglog(freq, response_bp_micboard)
plt.scatter(bp_freqs, bp_p2p_observed)
plt.legend()
plt.show()

<span style="color:#075a04"> **Sanity check: Does the frequency response of your band-pass filtered mic board match the frequency response of the band-pass filter multiplied with our speaker-mic board frequency response? Why or why not?** </span>

<a id='checkoff'> </a>

# <span style="color:#ba190f">CHECKOFF</span>


- **Have all questions and plots completed in this notebook.**

- **Show your TA your working circuits:**
    - Demonstrate working low-pass filter. 
    - Show your frequency response plots of the low-pass filter. 
    - Demonstrate working band-pass filter. 
    - Show your frequency response plots of the band-pass filter.

- **Be prepared to answer conceptual questions about the lab.**


**After checkoff, please:**

- Disassemble the high-pass filter.<span style="color:#ff0000"> **DO NOT DISMANTLE ANY OTHER CIRCUITS FROM YOUR BREADBOARD! You will need them for the rest of the project!**</span> 

- Store all your electrical components in a neat and organized manner.
- Make sure that all components that you received in your lab kit are stored safely for use in future labs.

- Remember what each part of your circuit is for (Record this in a Google doc or somewhere else safe). You will need to write summaries for both the midterm and final lab reports!
